In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

import sys

# setting path
sys.path.append('..')

from evaluation import get_time_step_fleet
from utils import load_solution

In [2]:
solution = load_solution("../data/solution_example.json")
df = get_time_step_fleet(solution=solution, ts=1)

In [3]:
df.columns

Index(['datacenter_id', 'server_generation', 'server_id', 'action'], dtype='object')

In [4]:
from utils import load_problem_data
demand, datacenters, servers, selling_prices = load_problem_data('../data')

In [5]:
demand_selected = demand[demand['time_step'] == 1]
demand_selected

,time_step,latency_sensitivity,CPU.S1,CPU.S2,CPU.S3,CPU.S4,GPU.S1,GPU.S2,GPU.S3
0,1,high,4000,0,0,0,30,0,0
168,1,medium,6000,0,0,0,10,0,0
336,1,low,10000,0,0,0,10,0,0


In [6]:
datacenters

,datacenter_id,cost_of_energy,latency_sensitivity,slots_capacity
0,DC1,0.25,low,25245
1,DC2,0.35,medium,15300
2,DC3,0.65,high,7020
3,DC4,0.75,high,8280


In [7]:
merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')

# Calculate Total Revenue (assuming selling price represents revenue per time unit over life expectancy)
merged_servers_df['total_revenue'] = merged_servers_df['selling_price'] * merged_servers_df['life_expectancy']

# Calculate Total Costs
merged_servers_df['total_cost'] = (
    merged_servers_df['purchase_price'] +
    (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
    merged_servers_df['cost_of_moving']
)

# Calculate Profitability
merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']

# Sort by Profitability
most_profitable_servers = merged_servers_df.sort_values(by='profitability', ascending=False)

most_profitable_servers

,server_generation,latency_sensitivity,selling_price,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee,total_revenue,total_cost,profitability
14,CPU.S1,high,25.0,CPU,"[1,60]",15000,2,400,60,96,1000,288,2400.0,43648,-41248.0
7,CPU.S1,medium,15.0,CPU,"[1,60]",15000,2,400,60,96,1000,288,1440.0,43648,-42208.0
0,CPU.S1,low,10.0,CPU,"[1,60]",15000,2,400,60,96,1000,288,960.0,43648,-42688.0
15,CPU.S2,high,25.0,CPU,"[37,96]",16000,2,460,75,96,1000,308,2400.0,46568,-44168.0
8,CPU.S2,medium,15.0,CPU,"[37,96]",16000,2,460,75,96,1000,308,1440.0,46568,-45128.0
1,CPU.S2,low,10.0,CPU,"[37,96]",16000,2,460,75,96,1000,308,960.0,46568,-45608.0
16,CPU.S3,high,27.5,CPU,"[73,132]",19500,2,800,120,96,1000,375,2640.0,56500,-53860.0
9,CPU.S3,medium,16.5,CPU,"[73,132]",19500,2,800,120,96,1000,375,1584.0,56500,-54916.0
2,CPU.S3,low,11.0,CPU,"[73,132]",19500,2,800,120,96,1000,375,1056.0,56500,-55444.0
17,CPU.S4,high,30.0,CPU,"[109,168]",22000,2,920,160,96,1000,423,2880.0,63608,-60728.0


In [8]:
import uuid


def generate_random_fleet(datacenters=datacenters, servers=servers, num_servers=100, max_time_step=168):
    # Create lists of valid datacenter IDs and server generations
    datacenter_ids = datacenters['datacenter_id'].tolist()
    server_generations = servers['server_generation'].tolist()

    # Generate random data
    random_fleet = pd.DataFrame({
        'datacenter_id': np.random.choice(datacenter_ids, num_servers),
        'server_generation': np.random.choice(server_generations, num_servers),
        'server_id': [str(uuid.uuid4()) for _ in range(num_servers)],
        'time_step_of_purchase': np.random.randint(1, max_time_step + 1, num_servers)
    })

    return random_fleet

In [9]:
fleet = generate_random_fleet()
fleet

,datacenter_id,server_generation,server_id,time_step_of_purchase
0,DC2,GPU.S2,b3ccfb9e-c04c-44a0-83a4-7b317993dc26,11
1,DC1,CPU.S2,806bb9fc-63cb-4713-9589-600a263054ea,149
2,DC4,CPU.S3,0b540545-da99-41e6-936f-0fde48b41e73,52
3,DC3,CPU.S1,d21e6a8c-3719-4afb-b659-2840e2709a09,53
4,DC4,CPU.S3,702165a5-0bf6-4427-bed3-4804f7ed2baf,8
...,...,...,...,...
95,DC3,GPU.S3,5630f974-1ec9-4b42-a613-9763be69baa4,116
96,DC1,GPU.S1,3894e474-6eea-41b3-853b-32db9205c0d3,89
97,DC3,CPU.S3,bfaa2400-7db7-491f-add5-b8face65a264,123
98,DC3,CPU.S2,f81df3f5-9805-4f5b-b601-f6de1e6064af,11


In [10]:
server_counts = fleet.groupby(['datacenter_id', 'server_generation']).size().reset_index(name="count")
server_counts

,datacenter_id,server_generation,count
0,DC1,CPU.S1,2
1,DC1,CPU.S2,2
2,DC1,CPU.S3,4
3,DC1,CPU.S4,3
4,DC1,GPU.S1,6
5,DC1,GPU.S2,1
6,DC1,GPU.S3,4
7,DC2,CPU.S1,5
8,DC2,CPU.S2,3
9,DC2,CPU.S3,3


In [11]:
fleet.loc[fleet["datacenter_id"] == "DC4"]

,datacenter_id,server_generation,server_id,time_step_of_purchase
2,DC4,CPU.S3,0b540545-da99-41e6-936f-0fde48b41e73,52
4,DC4,CPU.S3,702165a5-0bf6-4427-bed3-4804f7ed2baf,8
7,DC4,GPU.S2,66ff94fd-872a-4bc7-b943-dc10c619f048,132
9,DC4,CPU.S2,e7a570c9-687b-42c5-b215-85f20a9e1b0c,135
12,DC4,CPU.S3,58cac498-b39d-4cfb-898c-9fc7c6f315f6,149
13,DC4,CPU.S3,57a32954-4f61-46a6-9a36-f1e2ece3d526,68
16,DC4,GPU.S3,da7debd0-7e93-4f30-be0a-846ca1e66139,131
20,DC4,CPU.S2,8e44643d-2af7-420b-94ff-7b5243be653d,73
23,DC4,GPU.S1,d84da59b-f905-4aeb-bb50-215814e84308,19
30,DC4,CPU.S4,94cf2b72-3d2c-4475-8424-00d8682a4a92,131


In [12]:
def get_server_ages(fleet, time_step):
    return fleet['time_step_of_purchase'].apply(lambda x: time_step - x)

In [13]:
get_server_ages(fleet, 168)

0     157
1      19
2     116
3     115
4     160
     ... 
95     52
96     79
97     45
98    157
99     94
Name: time_step_of_purchase, Length: 100, dtype: int64

In [14]:
def generate_possible_actions(state, servers_info):
    buy_actions = []
    other_actions = []
    
    # Consider buying new servers
    for dc in state.datacenter_capacity.itertuples():
        for server in servers_info.itertuples():
            if state.time_step in eval(server.release_time):
                if dc.slots_capacity - dc.used_slots >= server.slots_size:
                    buy_actions.append({
                        'action': 'buy',
                        'datacenter_id': dc.datacenter_id,
                        'server_generation': server.server_generation,
                        'server_id': None  # We'll generate IDs when actually buying
                    })
    
    # Consider dismissing servers
    for server in state.fleet.itertuples():
        other_actions.append({
            'action': 'dismiss',
            'datacenter_id': server.datacenter_id,
            'server_generation': server.server_generation,
            'server_id': server.server_id
        })
    
    # Consider moving servers
    for server in state.fleet.itertuples():
        for dc in state.datacenter_capacity.itertuples():
            if dc.datacenter_id != server.datacenter_id:
                if dc.slots_capacity - dc.used_slots >= servers_info.loc[servers_info['server_generation'] == server.server_generation, 'slots_size'].values[0]:
                    other_actions.append({
                        'action': 'move',
                        'datacenter_id': dc.datacenter_id,
                        'server_generation': server.server_generation,
                        'server_id': server.server_id
                    })
    
    return buy_actions, other_actions

In [15]:
from evaluation import get_actual_demand, get_time_step_demand, get_capacity_by_server_generation_latency_sensitivity, get_utilization, get_normalized_lifespan, get_profit, change_selling_prices_format

def calculate_objective(fleet: pd.DataFrame, demand: pd.DataFrame, selling_prices: pd.DataFrame, time_step: int):
    if fleet.empty:
        return None
    
    fleet = fleet.merge(servers, on='server_generation')
    fleet = fleet.merge(datacenters, on='datacenter_id')
    
    # Calculate U (utilization)
    D = get_time_step_demand(get_actual_demand(demand), time_step)
    Zf = get_capacity_by_server_generation_latency_sensitivity(fleet)
    U = get_utilization(D, Zf)

    # Calculate L (normalized lifespan)
    L = get_normalized_lifespan(fleet)

    # Calculate P (profit)
    selling_prices = change_selling_prices_format(selling_prices)
    P = get_profit(D, 
                   Zf, 
                   selling_prices,
                   fleet)

    return U * L * P

In [16]:
from system_state import SystemState 

test_state = SystemState(datacenters, servers)
test_fleet = test_state.fleet.merge(servers, on='server_generation')
# test_fleet = test_fleet.merge(datacenters, on='datacenter_id')

test_fleet

,datacenter_id,server_generation,server_id,lifespan,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee


In [17]:
import copy
import system_state
import uuid

def generate_unique_id():
    return str(uuid.uuid4())

def greedy_decisions(state, demand, servers_info, selling_prices, datacenters):
    decisions = []
    buy_actions, other_actions = generate_possible_actions(state, servers_info)
    
    # First, handle non-buy actions
    while other_actions:
        best_action = None
        best_objective = calculate_objective(state.fleet, demand, selling_prices, state.time_step)
        
        for action in other_actions:
            temp_state = copy.deepcopy(state)
            temp_state.update_state([action])
            new_objective = calculate_objective(temp_state.fleet, demand, selling_prices, state.time_step)
            
            if new_objective > best_objective:
                best_objective = new_objective
                best_action = action
        
        if best_action:
            decisions.append(best_action)
            state.update_state([best_action])
            other_actions = [a for a in other_actions if a['server_id'] != best_action['server_id']]
        else:
            break
    
    # Now, handle buy actions
    while buy_actions:
        best_action = None
        best_objective = calculate_objective(state.fleet, demand, selling_prices, state.time_step)
        best_count = 0
        
        for action in buy_actions:
            dc_id = action['datacenter_id']
            dc_capacity = datacenters.loc[datacenters['datacenter_id'] == dc_id, 'slots_capacity'].values[0]
            dc_used_slots = state.datacenter_capacity.loc[state.datacenter_capacity['datacenter_id'] == dc_id, 'used_slots'].values[0]
            server_size = servers_info.loc[servers_info['server_generation'] == action['server_generation'], 'slots_size'].values[0]
            
            max_possible_servers = (dc_capacity - dc_used_slots) // server_size
            
            for count in range(1, max_possible_servers + 1):
                temp_actions = [{**action, 'server_id': generate_unique_id()} for _ in range(count)]
                
                temp_state = copy.deepcopy(state)
                temp_state.update_state(temp_actions)
                new_objective = calculate_objective(temp_state.fleet, demand, selling_prices, state.time_step)
                
                if new_objective > best_objective:
                    best_objective = new_objective
                    best_action = action
                    best_count = count
        
        if best_action:
            new_purchases = [{**best_action, 'server_id': generate_unique_id()} for _ in range(best_count)]
            decisions.extend(new_purchases)
            state.update_state(new_purchases)
            
            # Update datacenter capacity
            dc_id = best_action['datacenter_id']
            server_size = servers_info.loc[servers_info['server_generation'] == best_action['server_generation'], 'slots_size'].values[0]
            state.datacenter_capacity.loc[state.datacenter_capacity['datacenter_id'] == dc_id, 'used_slots'] += best_count * server_size
            
            # Remove the action if datacenter is full
            if state.datacenter_capacity.loc[state.datacenter_capacity['datacenter_id'] == dc_id, 'used_slots'].values[0] == datacenters.loc[datacenters['datacenter_id'] == dc_id, 'slots_capacity'].values[0]:
                buy_actions = [a for a in buy_actions if a['datacenter_id'] != dc_id]
        else:
            break
    
    return decisions

In [18]:
state = system_state.SystemState(datacenters, servers)

In [19]:
print(state.time_step)

1


In [20]:
greedy_decisions(state, demand, servers, selling_prices, datacenters)

/Users/artem/Desktop/projects/Hackathon-Huawei/analyse_data/../evaluation.py:294: RuntimeWarning: divide by zero encountered in log2
  return b * (1 + (((1.5)*(x))/xhat * np.log2(((1.5)*(x))/xhat)))
/Users/artem/Desktop/projects/Hackathon-Huawei/analyse_data/../evaluation.py:294: RuntimeWarning: invalid value encountered in scalar multiply
  return b * (1 + (((1.5)*(x))/xhat * np.log2(((1.5)*(x))/xhat)))


KeyError: 'moved'